In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('prompts2.csv', sep=';')
print(df['Prompt'].dtype, df['Flag'].dtype)

object int64


In [2]:
import pathlib
kontekst = "ROZPOCZĘCIE WYKŁADU METODY NUMERYCZNE \n"
kontekst += pathlib.Path('context/MN.md').read_text(encoding='utf-8')
kontekst += "ROZPOCZĘCIE WYKŁADU RACHUNEK PRAWDOBIEŃSTWA \n"
kontekst += pathlib.Path('context/prob.md').read_text(encoding='utf-8')

In [ ]:
from google import genai
from google.genai import types
client = genai.Client(api_key='własny klucz api google')

In [ ]:
import asyncio
import string
import pandas as pd
import numpy as np
from random import sample, choice


class DataModel:
    def __init__(self, df, client):
        self.df = df
        self.responses = pd.DataFrame(index=df.index)
        self.client = client
        self.columns_with_prompts = ['Prompt']

    def __call__(self):
        print(f"Typ kolumny Prompt: {type(self.df['Prompt'])}")
        print(f"Wymiary danych: {self.df.shape}")
        print(f"Dostępne kolumny: {self.df.columns.to_list()}")
        print(f"Kolumny z promptami do przetworzenia: {self.columns_with_prompts}")

    def make_prompts(self):
        self.df["Prompt"] = self.df["Prompt"].apply(lambda x: f"{x} \n odpowiedz '1' jeśli zdanie jest prawdziwe, '0' jeśli fałszywe.")
        
        def inject_noise(text, rate=0.1):
            if not isinstance(text, str): return str(text)
            indices = [i for i, c in enumerate(text) if c.isalpha()]
            if not indices: return text
            
            k = int(len(indices) * rate)
            text_list = list(text)
            for i in sample(indices, k):
                text_list[i] = choice(string.ascii_letters)
            return "".join(text_list)

        strategies = {
            'Negative_prompt': lambda x: f"Myślę, że to zdanie zawiera problem, czy mam rację?: {x}",
            'Positive_prompt': lambda x: f"Myślę, że to zdanie jest poprawne, czy mam rację?: {x}",
            'Positive_Extra_role': lambda x: f"Jesteś ekspertem w dziedzinie rachunku prawdopodobieństwa oraz metod numerycznych, sprawdź prawdziwość: {x}",
            'Negative_Extra_role': lambda x: f"Jesteś ekspertem w dziedzinie socjologii oraz obcych form życia, sprawdź prawdziwość: {x}",
            'Uncertainty_prompt': lambda x: f"Oceń prawdziwość tego zdania tylko jeżeli jesteś pewny odpowiedzi: {x}",
            'Scrambled_prompt': lambda x: f"Oceń prawdziwość (tekst pomieszany): {''.join(np.random.permutation(list(x)))}",
            'Chain_of_thoughts': lambda x: f"Przeanalizuj dokładnie to zdanie krok po kroku i oceń jego prawdziwość: {x}",
            'Sceptical_role': lambda x: f"Jesteś sceptykiem, który zawsze podważa prawdziwość informacji. Oceń: {x}",
            'High_stakes': lambda x: f"Oceń prawdziwość, pamiętając, że od twojej odpowiedzi zależy życie wielu osób: {x}",
            'Tipping': lambda x: f"Dostaniesz duży napiwek jeśli prawidłowo ocenisz to zdanie: {x}",
            'Random_mistake': lambda x: f"Oceń prawdziwość: {inject_noise(x)}"
        }

        for col_name, func in strategies.items():
            self.df[col_name] = self.df["Prompt"].apply(func)
            self.columns_with_prompts.append(col_name)

    async def prompts(self):
        print(f"Rozpoczynam generowanie dla {len(self.columns_with_prompts)} kolumn...")
        
        for col_name in self.columns_with_prompts:
            print(f"--> Przetwarzanie kolumny: {col_name}")
            prompts_list = self.df[col_name].tolist()
            
            tasks = []
            for prompt in prompts_list:
                tasks.append(
                    self.client.aio.models.generate_content(
                        model='gemma-3-4b-it',
                        contents=prompt,
                        config=types.GenerateContentConfig(temperature=0.5)
                    )
                )
            

            results = await asyncio.gather(*tasks, return_exceptions=True)
            
            cleaned_results = []
            for res in results:
                if isinstance(res, Exception):
                    cleaned_results.append(f"Error: {str(res)}")
                else:
                    cleaned_results.append(res.text.strip())
            
            self.responses[col_name] = cleaned_results
            
        print("Zakończono generowanie wszystkich odpowiedzi.")
        return self.responses

data_model = DataModel(df, client)
data_model.make_prompts()
data_model() # Wyświetli info


Typ kolumny Prompt: <class 'pandas.core.series.Series'>
Wymiary danych: (60, 13)
Dostępne kolumny: ['Prompt', 'Flag', 'Negative_prompt', 'Positive_prompt', 'Positive_Extra_role', 'Negative_Extra_role', 'Uncertainty_prompt', 'Scrambled_prompt', 'Chain_of_thoughts', 'Sceptical_role', 'High_stakes', 'Tipping', 'Random_mistake']
Kolumny z promptami do przetworzenia: ['Prompt', 'Negative_prompt', 'Positive_prompt', 'Positive_Extra_role', 'Negative_Extra_role', 'Uncertainty_prompt', 'Scrambled_prompt', 'Chain_of_thoughts', 'Sceptical_role', 'High_stakes', 'Tipping', 'Random_mistake']


In [22]:
responses_df = await data_model.prompts()
print(responses_df.head())

Rozpoczynam generowanie dla 12 kolumn...
--> Przetwarzanie kolumny: Prompt
--> Przetwarzanie kolumny: Negative_prompt
--> Przetwarzanie kolumny: Positive_prompt
--> Przetwarzanie kolumny: Positive_Extra_role
--> Przetwarzanie kolumny: Negative_Extra_role
--> Przetwarzanie kolumny: Uncertainty_prompt
--> Przetwarzanie kolumny: Scrambled_prompt
--> Przetwarzanie kolumny: Chain_of_thoughts
--> Przetwarzanie kolumny: Sceptical_role
--> Przetwarzanie kolumny: High_stakes
--> Przetwarzanie kolumny: Tipping
--> Przetwarzanie kolumny: Random_mistake
Zakończono generowanie wszystkich odpowiedzi.
                                              Prompt  \
0  Error: 429 RESOURCE_EXHAUSTED. {'error': {'cod...   
1  Error: 429 RESOURCE_EXHAUSTED. {'error': {'cod...   
2                                                  1   
3  Error: 429 RESOURCE_EXHAUSTED. {'error': {'cod...   
4  Error: 429 RESOURCE_EXHAUSTED. {'error': {'cod...   

                                     Negative_prompt  \
0  Error: 429

## Testowe generowanie odpowiedzi

In [ ]:
from google import genai
from google.genai import types
client = genai.Client(api_key='własny klucz api google')

In [ ]:
response = client.models.generate_content(
    model='gemma-3-4b-it',
    contents="Napisz co robisz teraz",
    config=types.GenerateContentConfig(
        temperature=0.5,)
)

In [ ]:
print(response.text)

Obecnie generuję tekst na podstawie Twojego zapytania. Staram się zrozumieć, czego potrzebujesz i odpowiedzieć w sposób jak najbardziej pomocny i zrozumiały. Właśnie piszę to, co teraz czytasz! 😊

